In [33]:

import requests
import hashlib
import time
import json as js
import pandas as pd
from pandas import json_normalize






    


In [34]:
secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page":1,
    "per_page":100
} 

total_pages = 21
# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}

data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # Process the response or store the data as per your requirement
    data = response.json()

    # Extract the relevant data records from the response
    
    data_records.extend(data['data'])

In [35]:
df = pd.DataFrame(data_records)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', 9,
                       'display.precision', 3,
                       ):
    print(df.head(10))

                     identifier                           name legality  \
0                eye-of-ophidia                 Eye of Ophidia       []   
1  dash-inventor-extraordinaire  Dash, Inventor Extraordinaire       []   
2                          dash                           Dash       []   
3           teklo-plasma-pistol            Teklo Plasma Pistol       []   
4           teklo-foundry-heart            Teklo Foundry Heart       []   
5          achilles-accelerator           Achilles Accelerator       []   
6                   high-octane                    High Octane       []   
7                    teklo-core                     Teklo Core       []   
8              maximum-velocity               Maximum Velocity       []   
9               spark-of-genius                Spark of Genius       []   

                              keywords  \
0             [generic, resource, gem]   
1                [mechanologist, hero]   
2         [mechanologist, hero, young]   
3  [me

In [39]:
df.dtypes

identifier    object
name          object
legality      object
keywords      object
stats         object
text          object
rarity        object
image         object
printings     object
dtype: object

In [43]:
df2 = df.astype({'stats':'string', 'image':'string'})
df3 = print(df2.apply(lambda row: row[df[['stats'],['rarity']].isin(['resource: 3'],['L', 'M'])]))

InvalidIndexError: (['stats'], ['rarity'])

InvalidIndexError: (['resource: 3'], ['L', 'M'])